# Text models, data, and training

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai import *
from fastai.docs import *

The [`text`](/text.html#text) module of the fastai library contains all the necessary functions to define a Dataset suitable for the various NLP (Natural Language Processing) tasks and quickly generate models you can use for them. Specifically:
- [`text.transform`](/text.transform.html#text.transform) contains all the scripts to preprocess your data, from raw text to token ids,
- [`text.data`](/text.data.html#text.data) contains the definition of [`TextDataset`](/text.data.html#TextDataset), which the main class you'll need in NLP,
- [`text.learner`](/text.learner.html#text.learner) contains helper functions to quickly create a language model or an RNN classifier.

Have a look at the links above for full details of the API of each module, of read on for a quick overview.

## Quick Start: Training an IMDb sentiment model with *ULMFiT*

Let's start with a quick end-to-end example of training a model. We'll train a sentiment classifier on a sample of the popular IMDb data, showing 4 steps:

1. Reading and viewing the IMDb data
1. Getting your data ready for modeling
1. Fine-tuning a language model
1. Building a classifier

### Reading and viewing the IMDb data

Contrary to images in Computer Vision, text can't directly be transformed into numbers to be fed into a model. The first thing we need to do is to preprocess our data so that we change the raw texts to lists of words, or tokens (a step that is called tokenization) then transform these tokens into numbers (a step that is called numericalization). These numbers are then passed to embedding layers that wil convert them in arrays of floats before passing them through a model.

You can find on the web plenty of [Word Embeddings](https://en.wikipedia.org/wiki/Word_embedding) to directly convert your tokens into floats. Those word embeddings have generally be trained on a large corpus such as wikipedia. Following the work of [ULMFiT](https://arxiv.org/abs/1801.06146), the fastai library is more focused on using pre-trained Language Models and fine-tuning them. Word embeddings are just vectors of 300 or 400 floats that represent different words, but a pretrained language model not only has those, but has also been trained to get a representation of full sentences and documents.

That's why the library is structured around three steps:

1. Get your data preprocessed and ready to use in a minimum amount of code,
1. Create a language model with pretrained weights that you can fine-tune to your dataset,
1. Create other models such as classifiers on top of the encoder of the language model.

To show examples, we have provided a small sample of the [IMDB dataset](https://www.imdb.com/interfaces/) which contains 1,000 reviews of movies with labels (positive or negative).

In [ ]:
untar_data(IMDB_PATH)
IMDB_PATH

PosixPath('../data/imdb_sample')

Creating a dataset from your raw texts is very simple if you have it in one of those ways
- organized it in folders in an ImageNet style
- organized in a csv file with labels columns and a text columns

Here, the sample from imdb is in a train and valid csv files that looks like this:

In [ ]:
df = pd.read_csv(IMDB_PATH/'train.csv', header=None)
df.head()

,0,1
0,0,Un-bleeping-believable! Meg Ryan doesn't even ...
1,1,This is a extremely well-made film. The acting...
2,0,Every once in a long while a movie will come a...
3,1,Name just says it all. I watched this movie wi...
4,0,This movie succeeds at being one of the most u...


Labels are encoded (though we can tell 1 seems to be positive from the bits of comments we see), and the file classes.txt contain the correspondence between index and names.

In [ ]:
classes = read_classes(IMDB_PATH/'classes.txt')
classes[0], classes[1]

('negative', 'positive')

### Getting your data ready for modeling

To create a dataset, we can just use the `TextDataset.from_csv` method. It will automatically do the two step of preprocessing for us. There is more information about those two steps in [`text.transform`](/text.transform.html#text.transform) where you'll also find how to customize the tokenizer from the fastai defaults. Note that to execute this line, you need to download the english model of spacy, which can be done by typing in your terminal:
```
python -m spacy download en
```

In [ ]:
for file in ['train_tok.npy', 'valid_tok.npy']:
    if os.path.exists(IMDB_PATH/'tmp'/file): os.remove(IMDB_PATH/'tmp'/file)

In [ ]:
train_ds = TextDataset.from_csv(IMDB_PATH, name='train', classes=classes)

Tokenizing train.


Numericalizing train.


Note that to avoid redoing this step (which can be quite time-consuming if you have a very large dataset), the fastai library has created a `tmp` directory in the folder given, to store the tokens (in `_tok.npy` files), the labels (in `_lbl.npy` files), the ids of the tokens (in `_id.npy` files) and the dictionary (`itos.pkl`). When you reexecute the line in the future, it will directly load that data.

To get a [`DataBunch`](/data.html#DataBunch) quickly, there are also several factory methods depending on how our data is structured. They are all detailed in [`transform.data`](/text.data.html#text.data), here we'll use [`text_data_from_csv`](/text.data.html#text_data_from_csv).

In [ ]:
# Language model data
data_lm = text_data_from_csv(Path(IMDB_PATH), data_func=lm_data)
# Classifier model data
data_clas = text_data_from_csv(Path(IMDB_PATH), data_func=classifier_data, vocab=data_lm.train_ds.vocab)

Tokenizing valid.


Numericalizing valid.


Note that [`TextDataset`](/text.data.html#TextDataset) was called behind the scenes for `train.csv` and `valid.csv` but, as explained earlier, it only preprocessed the validation set, since it had stored the result for the training set. The `data_func` argument here tells the [`text_data_from_csv`](/text.data.html#text_data_from_csv) function how to organize the data. In the first one, we prepare it for a language model, and in the second one, for a classifier (see the differences in [`text.data`](/text.data.html#text.data)).

For the classifier, we also pass the vocabulary (mapping from ids to words) that we want to use: this is to ensure that `data_clas` will use the same dictionary as `data_lm`.

### Fine-tuning a language model

We can use the `data_lm` object we created earlier to fine-tune a pretrained language model. [fast.ai](http://www.fast.ai/) has an English model available that we can download.

In [ ]:
download_wt103_model()

Now we can create a learner object that will directly create a model, load the pretrained weights and be ready for fine-tuning.

In [ ]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

Total time: 00:04
epoch  train loss  valid loss  accuracy
0      4.699907    4.201264    0.247246  (00:04)



Like a computer vision model, we can then unfreeze the model and fine-tune it.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

Total time: 00:27
epoch  train loss  valid loss  accuracy
0      4.464529    4.121362    0.255822  (00:05)
1      4.350247    4.043319    0.261671  (00:05)
2      4.229458    4.010417    0.264702  (00:05)
3      4.141184    3.991524    0.266229  (00:05)
4      4.082627    3.987816    0.265016  (00:05)



And finally we save the encoder to be able to use it for classification in the next section.

In [ ]:
learn.save_encoder('ft_enc')

### Building a classifier

We now use the `data_clas` object we created earlier to build a classifier with our fine-tuned encoder. The learner object can be done in a single line.

In [ ]:
learn = RNNLearner.classifier(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)

Total time: 00:05
epoch  train loss  valid loss  accuracy
0      0.591999    0.632481    0.620000  (00:05)



Again, we can unfreeze the model and fine-tune it.

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

Total time: 00:05
epoch  train loss  valid loss  accuracy
0      0.537493    0.570067    0.685000  (00:05)



In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

Total time: 00:09
epoch  train loss  valid loss  accuracy
0      0.436120    0.544485    0.735000  (00:09)

